<a href="https://www.kaggle.com/code/harshpt/finetuning-bloom-560m?scriptVersionId=226994469" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cp-train/Train.csv
/kaggle/input/cp-test/Test.csv


In [2]:
pip install --quiet peft datasets transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)
print("model done!")

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

model done!


In [5]:
print(foundation_model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [6]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

train_path = "/kaggle/input/cp-train/Train.csv"

train_df = pd.read_csv(train_path)

train_df = train_df[["Id", "long_title"]].dropna()

def preprocess_function(examples):
    text_input = examples['text']
    tokenized_input = tokenizer(text_input, truncation=True, padding="max_length", max_length=512)
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

train_df = train_df.rename(columns={"long_title": "text"})

train_dataset = Dataset.from_pandas(train_df)

train_dataset = train_dataset.map(preprocess_function, batched=True,remove_columns=['text'])

print("Training and Testing datasets are ready for fine-tuning!",train_dataset)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Training and Testing datasets are ready for fine-tuning! Dataset({
    features: ['Id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})


In [7]:
import peft
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=1,
    target_modules=["query_key_value"],
    lora_dropout=0.05, 
    bias="none",  
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 98,304 || all params: 559,312,896 || trainable%: 0.0176
None


In [8]:
mkdir cache

In [9]:
import transformers
from transformers import TrainingArguments, Trainer
import os

output_directory = os.path.join("../cache/working", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate= 2e-4, 
    num_train_epochs=1,
    no_cuda=False
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


Step,Training Loss
500,2.560100
1000,2.203100


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1250, training_loss=2.33919130859375, metrics={'train_runtime': 1885.6643, 'train_samples_per_second': 2.652, 'train_steps_per_second': 0.663, 'total_flos': 4652476951166976.0, 'train_loss': 2.33919130859375, 'epoch': 1.0})

In [10]:
model_save_path = "./bloom_lora_finetuned_new"
trainer.save_model(model_save_path)

In [11]:

model_path = "./bloom_lora_finetuned_new"

model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=1, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=1, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )

In [12]:
TEST_CSV = "/kaggle/input/cp-test/Test.csv"
 
def generate_response(prompt):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)  # Move input to GPU
    model.to(device)  
 
    output = model.generate(input_ids,max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)
 
 
# Example Usage
test_df = pd.read_csv(TEST_CSV)

queries = test_df['question'].tolist()
# print(queries)
# test_dataset = Dataset.from_pandas(test_df)

template = """répondez à la question suivante.
              {}"""
 
# sample = test_dataset[0]['question']
# formatted_prompt = template.format(sample)
 
# response = generate_response(formatted_prompt)

# print(f'Question: {sample}')
# print(f'Generated Response: {response.split("?")[1]}')

test_df["Model Response"] = [generate_response(template.format(query)).split("?")[1] for query in queries]
print(test_df["Model Response"])

0      \n              - Le régime de protection des ...
1      \n              1. Le bail doit être conclu pa...
2      \n              Ai-je droit à l'aide juridique...
3      \n              Oui, vous pouvez saisir la voi...
4      \n              - Les décisions relatives à la...
                             ...                        
215    \n              Je vous prie de bien vouloir r...
216    \n              Je vais être jugé par un juge ...
217    \n              Le divorce par consentement mu...
218    \n              1. La rupture de l’union conju...
219    \n              Pour les logements situés dans...
Name: Model Response, Length: 220, dtype: object


In [13]:
import time
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import precision_score, recall_score
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz
import logging


legal_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)

def generate_response(query):
    inputs = tokenizer(query, return_tensors="pt").to(device)
    start_time = time.time()
    output = model.generate(**inputs, max_length=100)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    latency = time.time() - start_time
    return response, latency

def calculate_fluency(response):
    words = response.split()
    total_words = len(words)
    errors = sum(1 for word in words if not word.isalpha())
    cwpm = total_words - errors
    return max(0, min(1, cwpm / total_words)) if total_words > 0 else 0

def compute_legal_accuracy(predictions, ground_truths):
    total_score = 0
    for pred, truth in zip(predictions, ground_truths):
        pred_embedding = legal_model.encode(pred, convert_to_tensor=True)
        truth_embedding = legal_model.encode(truth, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(pred_embedding, truth_embedding).item()
        total_score += similarity
    return max(0, min(1, total_score / len(predictions)))  # Normalize

def calculate_completeness(response, expected_length=100):
    response_length = len(response.split())
    return max(0, min(1, response_length / expected_length))  # Normalize

def calculate_relevance(response, query):
    relevance_score = fuzz.partial_ratio(response, query) / 100
    return max(0, min(1, relevance_score))  # Normalize

def evaluate_model(test_queries, reference_responses):
    avg_fluency, avg_accuracy, avg_completeness, avg_relevance, avg_latency = 0, 0, 0, 0, 0
    results = []

    for i, query in enumerate(test_queries):
        response, latency = generate_response(query)
        fluency = calculate_fluency(response)
        accuracy = compute_legal_accuracy([response], [reference_responses[i]])
        completeness = calculate_completeness(response)
        relevance = calculate_relevance(response, query)

        avg_fluency += fluency
        avg_accuracy += accuracy
        avg_completeness += completeness
        avg_relevance += relevance
        avg_latency += latency

    num_queries = len(test_queries)

    results.append([
        0,  # Placeholder for Unique Query ID
        avg_fluency / num_queries, 
        avg_accuracy / num_queries,
        avg_completeness / num_queries,
        avg_relevance / num_queries,
        avg_latency / num_queries
    ])

    return results

test_queries = test_df['question'].to_list()  
reference_responses = test_df["Model Response"].to_list()  

results = evaluate_model(test_queries, reference_responses)

# Convert to DataFrame and Save as CSV
import pandas as pd
df_results = pd.DataFrame(results, columns=["Unique_Query_ID", "Language_Fluency_Score", "Legal_Accuracy_Rating", "Response_Completeness", "Query_Relevance_Score", "Latency_Penalty_value"])
df_results.to_csv("submission.csv", index=False)

print("Evaluation Completed. Results saved to submission.csv")


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluation Completed. Results saved to submission.csv


In [14]:
res = pd.read_csv("/kaggle/working/submission.csv")
print(res)

   Unique_Query_ID  Language_Fluency_Score  Legal_Accuracy_Rating  \
0                0                0.810981               0.574916   

   Response_Completeness  Query_Relevance_Score  Latency_Penalty_value  
0                 0.6565                    1.0               1.555466  
